# BERT를 활용한 관계추출(Relation Extraction, RE)
본 Workspace에서는 klue/bert-base모델을 이용하여 KLUE 내의 8개 Task 중 관계추출(Relation Extraction) Task에 대해서 다룹니다.

먼저 관계추출 Task란 문장에 있는 두 개체(entity)간의 관계가 무엇인지 분류하는 것입니다.

따라서 문장과 문장에 있는 두 개체들이 입력으로 주어지면 두 개체들간의 관계가 출력으로 나옵니다.

![Imgur](https://i.imgur.com/xeTQRVC.png)

사진과 같이 개체는 subject entity와 object entity가 있는데, subject entity가 사람(person)이면 ":" 앞에는 per, 기관(organization)이면 org이 됩니다.

그리고 ":" 뒤에 있는 것은 object entity가 subject entity와 무슨 관계인지를 나타냅니다.

위의 사진과 같이 subject entity가 사람(person)이고, object entity가 subject entity의 출생지(origin)이므로 관계는 per:origin이 됩니다.

KLUE 관계추출 Task에 있는 관계는 총 30개이고, 관계일부를 캡처한 사진은 다음과 같습니다.

![Imgur](https://i.imgur.com/TnfSUPo.png)

만약 전체 관계 목록을 보고 싶으시면 <a href="https://github.com/KLUE-benchmark/KLUE/blob/main/klue_benchmark/klue-re-v1/relation_list.json">링크</a>를 확인해보세요!

# 필요한 라이브러리를 설치합니다.
datasets은 KLUE 데이터셋을 가져오기 위해, sklearn은 학습한 모델을 평가하기 위해 필요합니다.

In [1]:
!pip install datasets
!pip install sklearn

# 필요한 라이브러리를 import 합니다.

In [2]:
import torch
import torch.nn as nn
import sklearn.metrics

from tqdm import tqdm
from datasets import load_dataset
from datasets.arrow_dataset import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import DataLoader

# GPU 사용을 위해 device를 설정합니다.

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# KLUE RE 데이터셋을 가져옵니다.

In [4]:
dataset = load_dataset('klue', 're')

Reusing dataset klue (/workspace/.cache/huggingface/datasets/klue/re/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90)


# 데이터셋은 train과 validation 데이터로 구성되어 있습니다. 
train 데이터들은 모델을 학습할 때 사용될 예정이고, validation 데이터들은 학습이 아닌 모델의 성능을 평가할 때 사용됩니다.

train 데이터와 validation 데이터의 구성은 동일하므로 train 데이터의 구성만 살펴보도록 하겠습니다.

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['guid', 'sentence', 'subject_entity', 'object_entity', 'label', 'source'],
        num_rows: 32470
    })
    validation: Dataset({
        features: ['guid', 'sentence', 'subject_entity', 'object_entity', 'label', 'source'],
        num_rows: 7765
    })
})

# 데이터 구성
각 데이터들은 다음과 같이 문장과 관계 추출에 사용될 2개의 개체(object_entity, subject_entity)와 두 개체간의 관계를 라벨로 가지고 있습니다.

In [6]:
dataset['train'][0]

{'guid': 'klue-re-v1_train_00000',
 'sentence': '〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.',
 'subject_entity': {'word': '비틀즈',
  'start_idx': 24,
  'end_idx': 26,
  'type': 'ORG'},
 'object_entity': {'word': '조지 해리슨',
  'start_idx': 13,
  'end_idx': 18,
  'type': 'PER'},
 'label': 0,
 'source': 'wikipedia'}

# train데이터에서 각 label의 수를 살펴보겠습니다.
출력 결과를 보시면 30개의 라벨에 대해서 불균형이 심한 것을 확인할 수 있습니다.

따라서 모델의 성능을 측정할 평가지표로 단순하게 Accuracy를 사용한다면 정확한 평가가 이루어질 수 없기 때문에 KLUE 논문에 따르면 평가지표로 F1 score와 AUPRC를 이용하였습니다.


또한 관계 없음(0번 라벨) 데이터가 많은 비중을 차지하고 있는데, 모델이 "관계 없음"을 예측하는 데에 많은 초점이 맞춰지지 않도록

관계 없음에 해당하는 데이터를 제외하고 관계가 있는 데이터들에 대해서만 F1 score을 계산합니다.

In [7]:
label_count = {}

for data in dataset['train']:
    label = data['label']
    if label not in label_count:
        label_count[label] = 1
    else:
        label_count[label] += 1

label_count = dict(sorted(label_count.items(), key=lambda x: x[0]))
label_count

{0: 9534,
 1: 66,
 2: 450,
 3: 1195,
 4: 1320,
 5: 1866,
 6: 420,
 7: 98,
 8: 380,
 9: 155,
 10: 4284,
 11: 48,
 12: 1130,
 13: 418,
 14: 166,
 15: 40,
 16: 193,
 17: 1234,
 18: 3573,
 19: 82,
 20: 1001,
 21: 520,
 22: 304,
 23: 136,
 24: 795,
 25: 190,
 26: 534,
 27: 139,
 28: 96,
 29: 2103}

# 개체 양 끝에 special token을 추가합니다.
KLUE 논문에 따르면 object 개체의 양 끝에는 \<obj\>, \</obj\>을, subject 개체의 양 끝에는 \<subj\>, \</subj\> 토큰을 추가하여 개체의 위치를 표시한 후에 모델의 입력으로 주어집니다.

따라서 데이터에 있는 entity index를 이용해서 해당 토큰을 추가해줍니다.

토큰 추가의 예시 사진은 다음과 같습니다. 사진에서 빨간색으로 표시된 토큰들이 추가가 되는 토큰들입니다.

![Imgur](https://i.imgur.com/gWNeyLv.png)

그리고 학습에 필요한 데이터는 문장과 라벨 정보이므로 해당 부분만 가져오도록 합니다.

In [8]:
def add_entity_tokens(sentence, object_entity, subject_entity):
    obj_start_idx, obj_end_idx = object_entity['start_idx'], object_entity['end_idx']
    subj_start_idx, subj_end_idx = subject_entity['start_idx'], subject_entity['end_idx']
    
    if obj_start_idx < subj_start_idx:
        new_sentence = sentence[:obj_start_idx] + '<obj>' + sentence[obj_start_idx:obj_end_idx+1] + '</obj>' + \
                       sentence[obj_end_idx+1:subj_start_idx] + '<subj>' + sentence[subj_start_idx:subj_end_idx+1] + \
                       '</subj>' + sentence[subj_end_idx+1:]
    else:
        new_sentence = sentence[:subj_start_idx] + '<subj>' + sentence[subj_start_idx:subj_end_idx+1] + '</subj>' + \
                       sentence[subj_end_idx+1:obj_start_idx] + '<obj>' + sentence[obj_start_idx:obj_end_idx+1] + \
                       '</obj>' + sentence[obj_end_idx+1:]
    
    return new_sentence


def read_klue_re(dataset):
    sentences = []
    labels = []
    
    if isinstance(dataset, Dataset):
        for data in dataset:
            sentence = add_entity_tokens(data['sentence'], data['object_entity'], data['subject_entity'])
            sentences.append(sentence)
            labels.append(data['label'])
    
    return sentences, labels

In [9]:
# train, validation데이터셋에서 sentence와 label만 저장.
train_sentences, train_labels = read_klue_re(dataset['train'])
val_sentences, val_labels = read_klue_re(dataset['validation'])

In [10]:
# 개체 토큰이 정상적으로 잘 추가됐는지 확인하기 위해 train 문장 5개만 출력.
for i, sentence in enumerate(train_sentences[:5]):
    print(sentence, '\n')

〈Something〉는 <obj>조지 해리슨</obj>이 쓰고 <subj>비틀즈</subj>가 1969년 앨범 《Abbey Road》에 담은 노래다. 

호남이 기반인 바른미래당·<obj>대안신당</obj>·<subj>민주평화당</subj>이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다. 

K리그2에서 성적 1위를 달리고 있는 <subj>광주FC</subj>는 지난 26일 <obj>한국프로축구연맹</obj>으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다. 

균일가 생활용품점 (주)<subj>아성다이소</subj>(대표 <obj>박정부</obj>)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다. 

<obj>1967</obj>년 프로 야구 드래프트 1순위로 <subj>요미우리 자이언츠</subj>에게 입단하면서 등번호는 8번으로 배정되었다. 



# klue/bert-base 모델을 사용할 예정이므로 모델에 맞는 tokenizer를 가져옵니다.

In [11]:
model_name = 'klue/bert-base'

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

# tokenizer를 이용한 토큰화 결과가 어떻게 나오는지 살펴보도록 하겠습니다.
예시 문장으로는 첫 번째 train 데이터의 문장을 이용하도록 하겠습니다.

In [13]:
ex_sentence = dataset['train'][0]['sentence']

In [14]:
ex_sentence

'〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.'

In [15]:
ex_encoding = tokenizer(ex_sentence,
                        max_length=128,
                        padding='max_length',
                        truncation=True)

토큰화 결과로 Bert모델의 입력으로 필요한 input_ids, token_type_ids, attention_mask가 나오는 것을 확인할 수 있습니다.

3가지 값에 대한 설명은 <a href="https://huggingface.co/transformers/glossary.html#attention-mask">링크</a>에서 확인할 수 있습니다.

In [16]:
ex_encoding

{'input_ids': [2, 168, 30985, 14451, 7088, 4586, 169, 793, 8373, 14113, 2234, 2052, 1363, 2088, 29830, 2116, 14879, 2440, 6711, 170, 21406, 26713, 2076, 25145, 5749, 171, 1421, 818, 2073, 4388, 2062, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

토큰화 된 문장을 다시 디코딩 해봄으로써 원본 문장을 얻을 수 있는지 확인해봅니다.

디코딩 결과를 살펴보면 원본 문장 뒤에 [PAD] 토큰을 통해 입력 토큰의 개수가 max_length가 되도록 맞춥니다.

In [17]:
tokenizer.decode(ex_encoding['input_ids'])

'[CLS] 〈 Something 〉 는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《 Abbey Road 》 에 담은 노래다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

# Special token 추가
위에서 개체에 맞게 4개의 토큰(\<obj\>, \</obj\>, \<subj\>, \</subj\>)을 문장에 추가해주었는데, 해당 토큰들을 tokenizer에 special token이라고 알려주지 않으면 추가한 토큰들은 일반 문자로 인식되어서 토큰화될 수 있습니다. 

따라서 토큰화가 되지 않도록 추가한 4개의 토큰을 tokenizer에 special token으로 추가 해 줍니다.

In [18]:
entity_special_tokens = {'additional_special_tokens': ['<obj>', '</obj>', '<subj>', '</subj>']}
num_additional_special_tokens = tokenizer.add_special_tokens(entity_special_tokens)

데이터로더 및 학습에 필요한 값들을 설정합니다.

In [19]:
# For Dataloader
batch_size = 8

# For model
num_labels = 30

# For train
learning_rate = 1e-5
weight_decay = 0.0
epochs = 3

# 학습에 이용할 데이터셋과 데이터로더를 만들어 줍니다.

In [20]:
class KlueReDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, sentences, labels, max_length=128):
        self.encodings = tokenizer(sentences,
                                   max_length=max_length,
                                   padding='max_length',
                                   truncation=True)
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = self.labels[idx]
        
        return item
    
    def __len__(self):
        return len(self.labels)

In [21]:
train_dataset = KlueReDataset(tokenizer, train_sentences, train_labels)
val_dataset = KlueReDataset(tokenizer, val_sentences, val_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# klue/bert-base 모델을 로드합니다.

저희가 다루는 관계추출 Task는 30개의 관계(클래스)를 분류하는 것이라고 할 수 있습니다.

이를 위해 [CLS] 토큰의 벡터를 출력의 차원이 30인 1개의 Linear Layer에 통과시켜서 30개의 클래스로 분류하는 모델을 만들겠습니다.

이를 간단히 사진으로 나타내면 다음과 같습니다.

![Imgur](https://i.imgur.com/qaUObkV.png)

모델을 로드할 때 Warning이 발생하는데 "모델에서 가중치가 초기화되지 않은 부분이 있으니 예측을 하기 위해서는 모델을 학습시킨 후 사용해야된다."라는 내용을 담고 있습니다.

저희는 이후에 모델을 fine-tuning할 것이기 때문에 해당 메세지를 신경쓰지 않아도 됩니다.

In [22]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

모델 구성의 마지막에 있는 classifier 부분을 보시면 출력 차원이 30으로 설정되어 있는 것을 확인할 수 있습니다.

In [23]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

# Bert Embedding Layer을 resize합니다.

Bert에는 토큰들의 id에 따른 임베딩 값을 반환하는 Embedding Layer가 존재합니다.

하지만 현재 Embedding Layer에는 위에서 추가한 4개의 토큰에 대한 정보가 반영되지 않았기 때문에 추가한 토큰들이 입력으로 주어질 경우 index error가 발생합니다. 

따라서 Bert의 Embedding resize해줍니다.

Resize를 하게되면 Embedding Layer의 input 차원이 32000에서 32004로 4만큼 증가합니다.

In [24]:
model.resize_token_embeddings(len(tokenizer))

Embedding(32004, 768)

학습 도중 Loss, Accuracy 계산 및 저장을 간단하게 하기 위해 AverageMeter를 클래스를 이용합니다.

In [25]:
class AverageMeter():
    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

# Model fine-tuning
klue/bert-base 모델을 fine-tuning합니다.

학습동안 학습이 잘 진행되고 있는지 확인하기 위해 Loss와 Accuracy를 출력합니다.

Tesla T4 기준으로 1 epoch 당 약 14분 정도가 소요됩니다.

In [26]:
def train_epoch(data_loader, model, criterion, optimizer, train=True):
    loss_save = AverageMeter()
    acc_save = AverageMeter()
    
    loop = tqdm(enumerate(data_loader), total=len(data_loader))
    for _, batch in loop:
        inputs = {
            'input_ids': batch['input_ids'].to(device),
            'token_type_ids': batch['token_type_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device),
        }
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        outputs = model(**inputs)
        logits = outputs['logits']
        
        loss = criterion(logits, labels)
        
        if train:
            loss.backward()
            optimizer.step()
            
        preds = torch.argmax(logits, dim=1)
        acc = ((preds == labels).sum().item() / labels.shape[0])
        
        loss_save.update(loss, labels.shape[0])
        acc_save.update(acc, labels.shape[0])
        
    results = {
        'loss': loss_save.avg,
        'acc': acc_save.avg,
    }
    
    return results
        
        
# loss function, optimizer 설정
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

for epoch in range(epochs):
    print(f'< Epoch {epoch+1} / {epochs} >')
    
    # Train
    model.train()
    
    train_results = train_epoch(train_loader, model, criterion, optimizer)
    train_loss, train_acc = train_results['loss'], train_results['acc']
    
    # Validation
    with torch.no_grad():
        model.eval()
        
        val_results = train_epoch(val_loader, model, criterion, optimizer, False)
        val_loss, val_acc = val_results['loss'], val_results['acc']
    
    
    print(f'train_loss: {train_loss:.4f}, train_acc: {train_acc:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}')
    print('=' * 100)

  0%|          | 0/4059 [00:00<?, ?it/s]

< Epoch 1 / 3 >


  0%|          | 0/4059 [00:00<?, ?it/s]

train_loss: 1.0405, train_acc: 0.6958, val_loss: 0.8702, val_acc: 0.6979
< Epoch 2 / 3 >


  0%|          | 0/4059 [00:00<?, ?it/s]

train_loss: 0.5159, train_acc: 0.8258, val_loss: 0.7869, val_acc: 0.7283
< Epoch 3 / 3 >


100%|██████████| 971/971 [01:08<00:00, 14.17it/s]

train_loss: 0.3583, train_acc: 0.8792, val_loss: 0.8018, val_acc: 0.7415


# 학습된  모델을 저장합니다.

In [27]:
tokenizer.save_pretrained('./klue-bert-base-re')
model.save_pretrained('./klue-bert-base-re')

# Validation 결과 확인
먼저 관계 없음에 해당하는 문장에 대한 예측 결과를 살펴보겠습니다.

In [28]:
val_sentence = val_sentences[0]

val_sentence

"20대 남성 <subj>A</subj>(26)씨가 아버지 치료비를 위해 B(<obj>30</obj>)씨가 모아둔 돈을 훔쳐 인터넷 방송 BJ에게 '별풍선'으로 쏜 사실이 알려졌다."

In [29]:
val_encoding = tokenizer(val_sentence,
                         max_length=128,
                         padding='max_length',
                         truncation=True,
                         return_tensors='pt')

In [30]:
val_input = {
    'input_ids': val_encoding['input_ids'].to(device),
    'token_type_ids': val_encoding['token_type_ids'].to(device),
    'attention_mask': val_encoding['attention_mask'].to(device),
}

model.eval()
output = model(**val_input)
label = torch.argmax(output['logits'], dim=1)

0번 라벨은 "관계 없음"이므로 해당 문장에 대해서 예측이 정확하게 됐음을 알 수 있습니다.

In [31]:
label

tensor([0], device='cuda:0')

다음으로 관계가 있는 문장에 대한 예측 결과를 살펴보겠습니다.

In [32]:
val_sentence = val_sentences[13]

val_sentence

'<subj>서울교통공사</subj>는 <obj>서울</obj> 노원구 석계역 무빙워크에 고의로 침을 바른 남성이 신종 코로나바이러스 감염증(코로나19) 검사에서 음성 판정이 나왔다고 21일 밝혔다.'

In [33]:
val_encoding = tokenizer(val_sentence,
                         max_length=128,
                         padding='max_length',
                         truncation=True,
                         return_tensors='pt')

In [34]:
val_input = {
    'input_ids': val_encoding['input_ids'].to(device),
    'token_type_ids': val_encoding['token_type_ids'].to(device),
    'attention_mask': val_encoding['attention_mask'].to(device),
}

model.eval()
output = model(**val_input)
label = torch.argmax(output['logits'], dim=1)

3번 라벨은 "org:place_of_headquarters" 이므로 해당 문장에 대해서 예측이 정확하게 됐음을 알 수 있습니다.

In [35]:
label

tensor([3], device='cuda:0')

# 모델 평가
학습된 모델에 대해서 평가를 해보도록 하겠습니다.

KLUE 논문에 따르면 평가 지표로는 F1 score, AUPRC가 사용되었지만 본 Workspace에서는 F1 score 성능만 측정해보도록 하겠습니다.

이 때, 0번 라벨(관계 없음)을 제외한 관계가 있는 라벨들에 대해서만 F1 score를 계산합니다.

In [36]:
def calc_f1_score(preds, labels):
    """
    label이 0(관계 없음)이 아닌 예측 값에 대해서만 f1 score 계산.
    """
    preds_relation = []
    labels_relation = []
    
    for pred, label in zip(preds, labels):
        if label != 0:
            preds_relation.append(pred)
            labels_relation.append(label)
    
    f1_score = sklearn.metrics.f1_score(labels_relation, preds_relation, average='micro', zero_division=1)
    
    return f1_score * 100

In [37]:
with torch.no_grad():
    model.eval()
    
    label_all = []
    pred_all = []
    for batch in tqdm(val_loader):
        inputs = {
            'input_ids': batch['input_ids'].to(device),
            'token_type_ids': batch['token_type_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device),
        }
        labels = batch['labels'].to(device)
        
        outputs = model(**inputs)
        logits = outputs['logits']
        
        preds = torch.argmax(logits, dim=1)
        
        label_all.extend(labels.detach().cpu().numpy().tolist())
        pred_all.extend(preds.detach().cpu().numpy().tolist())
    
    f1_score = calc_f1_score(label_all, pred_all)

100%|██████████| 971/971 [01:07<00:00, 14.32it/s]


약 61.93의 F1 score를 기록했습니다. 하이퍼파라미터, 모델 구조 등을 변경시켜 다양한 학습을 시도해보세요!

In [38]:
f1_score

61.93164001254312

학습된 모델은 <a href="https://huggingface.co/ainize/klue-bert-base-re">Huggingface Model Hub</a>에 배포되어 있고, 언제든지 다운받아서 활용이 가능합니다.

모델 로드 후 Inference 방법은 해당 링크에 있는 설명부분에 작성되어 있습니다.

# Reference

### <a href="https://github.com/Huffon/klue-transformers-tutorial/blob/master/sentence_transformers.ipynb">klue-transformers-tutorial</a>

### <a href="https://huggingface.co/transformers/custom_datasets.html">Fine-tuning with custom datasets</a>